# Практическое занятие №3

П.Н. Советов, РТУ МИРЭА

**Задача 1**

Приведите примеры кода, которые соответствуют следующим нарушениям PEP 8:

1. whitespace before '('.
1. missing whitespace around operator.
1. missing whitespace after ','.
1. unexpected spaces around keyword / parameter equals.
1. expected 2 blank lines, found 1.
1. multiple statements on one line (colon).
1. multiple statements on one line (semicolon).
1. comparison to None should be 'if cond is None:'.
1. comparison to True should be 'if cond is True:' or 'if cond:'.

Для быстрой проверки используйте сайт [pep8online.com](http://pep8online.com).

In [1]:
# 1.
def function (a):
    return a

# 5.
def function (b):
    return b

# 2.
x =y= 0

# 3.
print(x,y)

# 4.
print(x or  y)

# 6.
if not x: print(x)
    
# 7.
print(x); print(y)

# 8.
if x == None:
    print(x)

# 9.
if x == True:
    print(x)

0 0
0
0
0
0


**Задача 2**

Как вы думаете, модуль загружается один раз или загружается каждый раз при очередном импорте? Докажите правильность вашей гипотезы кодом.

In [2]:
# Спасибо Егору П. (https://github.com/eiparfenov) за решение.

In [3]:
# number.py
print("imported")

imported


**Задача 8**

Изобразите легендарную первую галактику из игры Elite (1984) с помощью Matplotlib. В этой игре целая вселенная описывалась с помощью всего трех чисел. Алгоритм процедурной генерации звезд, а точнее их расположений и имен, можно найти [на этом сайте](http://blog.rabidgremlin.com/2015/01/14/procedural-content-generation-creating-a-universe/). Описание первой галактики можно найти [здесь](https://wiki.alioth.net/index.php/Oolite_planet_list/Galaxy_1).

In [ ]:
import matplotlib.pyplot as plt


BASE_0 = 0x5A4A  # число 23114
BASE_1 = 0x0248  # число 584
BASE_2 = 0xB753  # число 46931


pairs = "..LEXEGEZACEBISOUSESARMAINDIREA.ERATENBERALAVETIEDORQUANTEISRION"
galaxy = []


def tweakseed(s):
    temp = s[0] + s[1] + s[2]
    s[0] = s[1]
    s[1] = s[2]
    s[2] = temp
    return s


def stripout(s, c):
    s = s.replace(c, '')
    return s


def makesystem(s):
    longnameflag = s[0] & 64
    thissys = [0, 0, '']
    thisname = [''] * 12
    thissys[0] = (s[1] >> 8) % 256
    thissys[1] = (s[0] >> 8) % 256
    pair1 = 2 * ((s[2] >> 8) & 31)
    s = tweakseed(s)
    pair2 = 2 * ((s[2] >> 8) & 31)
    s = tweakseed(s)
    pair3 = 2 * ((s[2] >> 8) & 31)
    s = tweakseed(s)
    pair4 = 2 * ((s[2] >> 8) & 31)
    s = tweakseed(s)
    thisname[0] = pairs[pair1]
    thisname[1] = pairs[pair1 + 1]
    thisname[2] = pairs[pair2]
    thisname[3] = pairs[pair2 + 1]
    thisname[4] = pairs[pair3]
    thisname[5] = pairs[pair3 + 1]
    if longnameflag:
        thisname[6] += pairs[pair4]
        thisname[7] += pairs[pair4 + 1]
    thisname = ''.join(thisname)
    thissys[2] = stripout(thisname, '.')
    return thissys


seed = [BASE_0, BASE_1, BASE_2]
for i in range(0, 256):
    galaxy.append(makesystem(seed))


xs = range(0, 256)
plt.title('First Galaxy')
for i in range(0, 256):
    plt.scatter(galaxy[i][0], 256 - galaxy[i][1], c = 'red')
    plt.text(galaxy[i][0], 256 - galaxy[i][1], galaxy[i][2])
plt.show()


**Задача 9**

Реализуйте модель сегрегации Шеллинга в Matplotlib. На двумерной сетке находятся агенты двух групп. На каждой клетке может находиться не более 1 агента. Агент «счастлив», если, как минимум, заданный количество ближайших соседей относится к его группе. В противном случае агент переезжает на иное, свободное место.

Ввести следующие параметры: размер популяции, размеры сетки, процентное соотношение агентов двух групп, пороговое значение «толерантности», количество шагов моделирования.

1. Реализовать отображение агентов в виде квадратов двух цветов на целочисленной сетке.
1. Случайно разместить агентов, учитывая запрет на совпадение координат.
1. Реализовать функцию `distance` на основе метрики манхэттенского расстояния.
1. Реализовать функцию `is_happy`.
1. Изобразить график исходного расположения агентов и график расположения спустя N шагов моделирования.
1. Изобразить график изменения состояния «настроения» агентов.
1. (повышенной сложности) Реализовать анимацию шагов моделирования.

In [ ]:
from random import randint, choice
import matplotlib.pyplot as plt
import time


class Agent:
    MAX = 20
    NEIGHBOURS = 10
    PERCENT = 0.8

    def __init__(self, group):
        self.x = randint(0, self.MAX)
        self.y = randint(0, self.MAX)
        self.group = group

    def distance(self, other: 'Agent') -> float:
        return abs(self.x - other.x) + abs(self.y - other.y)

    def happy(self, others):
        neighbours = 0
        ms = []
        for other in others:
            ms.append((self.distance(other), other))

        ms.sort(key=lambda x: x[0])
        for i in range(self.NEIGHBOURS):
            if ms[i][1].group == self.group:
                neighbours += 1

        return neighbours >= self.NEIGHBOURS * self.PERCENT

    def update(self, others):
        if not self.happy(others):
            while True:
                nx, ny = randint(0, self.MAX), randint(0, self.MAX)
                for other in others:
                    if other.x == nx and other.y == ny:
                        continue
                self.x, self.y = nx, ny
                break


def simulation(grs, field, k, p):

    Agent.PERCENT = p
    Agent.MAX = field
    Agent.NEIGHBOURS = k

    groups = ['first', 'second']

    agents = [Agent(groups[0]) for i in range(grs[0])] + [Agent(groups[1]) for i in range(grs[1])]
    happy = []

    plt.ion()

    for i in range(20):
        c_h = 0
        for agent in agents:
            agent.update(agents)
            if agent.happy(agents):
                c_h += 1
        if c_h == len(agents):
            break
        happy.append(c_h / len(agents))
        x = [[], []]
        y = [[], []]
        for agent in agents:
            group = groups.index(agent.group)
            x[group].append(agent.x)
            y[group].append(agent.y)

        plt.clf()
        print(x[0], y[0])
        plt.plot(x[0], y[0], 's', markerfacecolor='darkblue', markersize=10)
        plt.plot(x[1], y[1], 's', markerfacecolor='darkgreen', markersize=10)
        '''
        ax = plt.gca()
        ax.set_facecolor('yellow')
        '''
        plt.draw()
        plt.gcf().canvas.flush_events()
        time.sleep(0.2)

    plt.ioff()
    plt.clf()
    plt.plot(happy)
    plt.show()


if __name__ == '__main__':
    simulation((80, 80), 20, 10, 0.8)
